# 使用GEE导入切片
日期选择、时序影像平均、降低噪声、几何畸变校正

In [ ]:
import ee
import geemap
import math
import sys,os
from GEE_Func.S1_distor_dedicated import load_S1collection,S1_CalDistor,DEM_caculator
from GEE_Func.S2_filter import merge_s2_collection
from GEE_Func.GEE_DataIOTrans import DataTrans
from GEE_Func.GEE_CorreterAndFilters import ImageFilter,S1Corrector
from GEE_Func.GEEMath import get_minmax
from GEE_Func.GEE_Tools import Select_imageNum 
from functools import partial
geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()

## 载入GEE冰湖矢量，获取相应范围的图像切片

In [ ]:
#--------------------------预加载冰湖数据,测试的时候加上Filter_bound
Filter_bound = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/Test/Domin_ForDistorTest').geometry().buffer(-300)
Glacial_lake = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/Glacial_lake/SAR_GLs/2019Gls_SARExt').sort('fid_1').filterBounds(Filter_bound)
Glacial_lake_Shrink = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/Glacial_lake/SAR_GLs/2019Gls_SARExt_inbuffer').sort('fid_1').filterBounds(Filter_bound)
DEMNASA = ee.Image("NASA/NASADEM_HGT/001").select('elevation')

Ascending_DistorFull = ee.Image('projects/ee-mrwurenzhe/assets/SETP_Distor/SETPAscending_Distor')
Descending_DistorFull= ee.Image('projects/ee-mrwurenzhe/assets/SETP_Distor/SETPDescending_Distor')
Ascending_GradFull   = ee.Image('projects/ee-mrwurenzhe/assets/SETP_Distor/SETPAscending_Forshortening_Grading_30')
Descending_GradFull  = ee.Image('projects/ee-mrwurenzhe/assets/SETP_Distor/SETPDescending_Forshortening_Grading_30')
restrict_Fuse = False # 图像融合方式
Filter_Angle  = 36    # 数字越小越容易导入Layover图像，普遍值为32-45
Nodata_tore   = 0     # 感兴趣区域nodata像素数容忍度
NodataTovalue = 0
Origin_scale = 10     # 原始数据分辨率
projScale = 30        # 投影分辨率
box_fromGEE = True    # box是否由GEE获得

# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()})
Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_C = Glacial_lake.map(Geo_ext)
Num_list = Glacial_lake.size().getInfo()
Glacial_lake_A_GeoList = Glacial_lake.toList(Num_list)
Glacial_lake_C_CentriodList = ee.List(Glacial_lake_C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_R_RectangleList = ee.List(Glacial_lake_C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Glacial_lake_Shrink_GeoList = Glacial_lake_Shrink.toList(Num_list)

In [ ]:
def getS1Corners(image, orbitProperties_pass):
    # 真实方位角(根据整幅影响运算)
    coords = ee.Array(image.geometry().coordinates().get(0)).transpose()
    crdLons = ee.List(coords.toList().get(0))
    crdLats = ee.List(coords.toList().get(1))
    minLon = crdLons.sort().get(0)
    minLat = crdLats.sort().get(0)
    azimuth = (ee.Number(crdLons.get(crdLats.indexOf(minLat))).subtract(minLon).atan2(
        ee.Number(crdLats.get(crdLons.indexOf(minLon))).subtract(minLat))
                .multiply(180.0 / 3.131415926))

    if orbitProperties_pass == 'ASCENDING':
        azimuth = azimuth.add(270.0)
    elif orbitProperties_pass == 'DESCENDING':
        azimuth = azimuth.add(180.0)
    else:
        raise TypeError
    
    return azimuth

def S1_slope_correction(image, orbitProperties_pass,DEM = DEMNASA,scale=Origin_scale, model='volume'):
    # 获取投影几何
    geom = image.geometry()
    proj = image.select(0).projection()

    # 计算方位向、距离向以及图像的四个角点、构筑计算辅助线
    Heading = getS1Corners(image,orbitProperties_pass) #'ASCENDING' = 348.3   'DESCENDING' = 189.2

    s1_azimuth_across = ee.Number(Heading).subtract(90.0) # 距离向
    theta_iRad = image.select('angle').multiply(3.1415926 / 180)  # 地面入射角度转为弧度
    phi_iRad = ee.Image.constant(s1_azimuth_across).multiply(3.1415926 / 180)  # 距离向转弧度

    #计算地形几何信息
    _, _, alpha_rRad, alpha_azRad = DEM_caculator.slop_aspect(DEM, proj, geom, phi_iRad)

    # 根据入射角度修订
    sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))
    gamma0 = sigma0Pow.divide(theta_iRad.cos())

    slop_correction = S1Corrector.volumetric(model, theta_iRad, alpha_rRad, alpha_azRad,gamma0)
    Eq_pixels = DataTrans.Eq_pixels
    image_ = (Eq_pixels(slop_correction['gamma0_flatDB'].select('VV_gamma0flat')).rename('VV_gamma0_flatDB')
            .addBands(Eq_pixels(slop_correction['gamma0_flatDB'].select('VH_gamma0flat')).rename('VH_gamma0_flatDB'))
            .addBands(Eq_pixels(image.select('angle')).rename('incAngle'))).reproject(crs=proj)
    
     # 手动复制时间信息
    time_start = image.get('system:time_start')
    time_end = image.get('system:time_end')
    image_ = image_.set('system:time_start', time_start).set('system:time_end', time_end)
    return image_.copyProperties(image)

def getMask(image, nodataValues: list = []):
    combined_mask = image.mask()  # 获取现有的掩膜
    if len(nodataValues) == 0:
        return combined_mask
    else:
        for nodatavalue in nodataValues:
            value_mask = image.neq(nodatavalue).unmask(0) 
            combined_mask = combined_mask.And(value_mask)  
        return combined_mask

In [ ]:
i = 15
GLAOI = ee.Feature(Glacial_lake_A_GeoList.get(i))
AOI = GLAOI.geometry()
AOI_Bound = ee.Feature(Glacial_lake_R_RectangleList.get(i)).geometry()
AOI_Centro = ee.Feature(Glacial_lake_C_CentriodList.get(i)).geometry()
AOI_area = float(GLAOI.get('Area_pre').getInfo())
AOI_bufferBounds = AOI_Bound.buffer(distance=math.log(AOI_area+1,5) * 1200 + 1200).bounds()# 边界不可过大，否则会影响局部自适应提取效果



years = ['2015','2016','2017','2018','2019','2020','2021','2022','2023','2024']
SETP_Season = ['-02-25','-05-31','-09-15','-11-28','-02-25']

START_DATE  = ee.Date(years[4] + SETP_Season[1])
END_DATE    = ee.Date(years[4] + SETP_Season[2])

s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
        .filterBounds(AOI)
        .filterDate(START_DATE, END_DATE))

s1_col = s1_col.map(partial(DataTrans.rm_nodata, AOI=AOI)).map(partial(DataTrans.cal_minmax,AOI=AOI))
s1_col = s1_col.filter(ee.Filter.eq('numNodata', Nodata_tore)).filter(ee.Filter.gt('min', 36)).map(ImageFilter.RefinedLee)
proj = s1_col.first().select(0).projection()

# 斜坡校正
s1_ascending_collection= s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
                                                .map(partial(S1_slope_correction,orbitProperties_pass='ASCENDING'))
s1_descending_collection = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
                                                .map(partial(S1_slope_correction,orbitProperties_pass='DESCENDING'))

# 数据集求均值，降低噪声、滩涂、流水干扰
s1_ascending = s1_ascending_collection.mean().reproject(crs=proj).clip(AOI_bufferBounds)
s1_descending = s1_descending_collection.mean().reproject(crs=proj,scale=Origin_scale).clip(AOI_bufferBounds)

# 计算几何畸变掩膜
Ascending_Distor = Ascending_DistorFull.clip(AOI_bufferBounds)
Descending_Distor = Descending_DistorFull.clip(AOI_bufferBounds) 
A_Mask = getMask(Ascending_Distor,nodataValues=[9])
D_Mask = getMask(Descending_Distor,nodataValues=[9])
Ascending_Grad = Ascending_GradFull.clip(AOI_bufferBounds)
Descending_Grad = Descending_GradFull.clip(AOI_bufferBounds) 
A_Mask = A_Mask.Or(Ascending_Grad.gt(10).unmask(0))
D_Mask = D_Mask.Or(Descending_Grad.gt(10).unmask(0))

In [ ]:
# timestamp = ee.Date(Select_imageNum(s1_ascending_collection,1).get('system:time_start')).format('YYYY-MM-dd HH:mm:ss').getInfo()
# print('Sentinel-1影像的时间:', timestamp)

# os.environ["PLANET_API_KEY"] = "PLAKc3ce7baa106b47d980fd28a59c3ede5b"
# tile_format = "ipyleaflet"
# if os.environ.get("USE_FOLIUM") is not None:
#     tile_format = "folium"

# Map = geemap.Map()
# Map.centerObject(AOI_Centro, zoom=15)
# Map.add_planet_by_quarter(year=2019, quarter=1)
# Map.add_planet_by_month(year=2019, month=3)

# Map.addLayer(s1_descending.select(0),{'min':-15,'max':-1},'a')
# Map.addLayer(Select_imageNum(s1_descending_collection,0).select(0).clip(AOI_bufferBounds),{'min':-15,'max':-1},'a_0')
# Map.addLayer(Select_imageNum(s1_descending_collection,1).select(0).clip(AOI_bufferBounds),{'min':-15,'max':-1},'a_1')
# Map.addLayer(Select_imageNum(s1_descending_collection,2).select(0).clip(AOI_bufferBounds),{'min':-15,'max':-1},'a_2')
# Map.addLayer(Select_imageNum(s1_descending_collection,3).select(0).clip(AOI_bufferBounds),{'min':-15,'max':-1},'a_3')
# Map.addLayer(Select_imageNum(s1_descending_collection,4).select(0).clip(AOI_bufferBounds),{'min':-15,'max':-1},'a_4')
# Map.addLayer(Select_imageNum(s1_descending_collection,5).select(0).clip(AOI_bufferBounds),{'min':-15,'max':-1},'a_5')
# Map

# 升降轨融合

In [ ]:
if restrict_Fuse:
        # 严格。如果A、D都存在畸变，则该区域为空
        s1_ascending_ =  s1_ascending.updateMask(A_Mask.Not())
        s1_descending_=  s1_descending.updateMask(D_Mask.Not())                      
        Combin_AD = ee.ImageCollection([s1_ascending_,s1_descending_])
        s1_unit_mean_ = Combin_AD.mean().reproject(crs=proj).clip(AOI_bufferBounds)

else:
        # 宽松，如果存在畸变，则优先采用逆轨影像替换
        s1_ascending_ = ee.Image(s1_ascending.where(A_Mask,s1_descending))
        s1_descending_ = ee.Image(s1_descending.where(D_Mask,s1_ascending))
        Combin_AD = ee.ImageCollection([s1_ascending_,s1_descending_])
        s1_unit_mean_ = Combin_AD.mean().reproject(crs=proj).clip(AOI_bufferBounds)
# 设置Nodata为-99
s1_unit_mean_ = s1_unit_mean_.unmask(NodataTovalue)

In [ ]:
# os.environ["PLANET_API_KEY"] = "PLAKc3ce7baa106b47d980fd28a59c3ede5b"
# tile_format = "ipyleaflet"
# if os.environ.get("USE_FOLIUM") is not None:
#     tile_format = "folium"
# quarterly_tiles = geemap.planet_quarterly_tiles(tile_format=tile_format)

# Map = geemap.Map()
# Map.centerObject(AOI_Centro, zoom=15)
# Map.add_planet_by_quarter(year=2019, quarter=3)
# Map.addLayer(A_Mask.randomVisualizer())
# Map.addLayer(s1_ascending_.select(0),{'min':-15,'max':-1},'a')
# Map.addLayer(D_Mask.randomVisualizer())
# Map.addLayer(s1_descending_.select(0),{'min':-15,'max':-1},'d')
# Map.addLayer(s1_unit_mean_.select(0),{'min':-15,'max':-1},'s1_unit_mean_')
# Map

## 图像切片导出，采用本地文件导入

### 导出升降轨融合图像

In [ ]:
import importlib
import GEE_Func.GEE_DataIOTrans
importlib.reload(GEE_Func.GEE_DataIOTrans)
from GEE_Func.GEE_DataIOTrans import DataIO
from PackageDeepLearn.utils import DataIOTrans
os.chdir(r'D:\Dataset_and_Demo\DataFused')
save_path = "{}_ADMeanFused.tif".format(f'{i:05d}')
# 数据导出，转为0-255
DataIO.Geemap_export(save_path, s1_unit_mean_,region=AOI_bufferBounds, scale=Origin_scale,rename_image=False)
#---------------------！！！！！！！！！！！！！！！！修改了nodata、transFunc再测试
imagePath = DataIOTrans.DataIO.TransImage_Values(save_path,transFunc=DataIOTrans.DataTrans.MinMaxBoundaryScaler,bandSave=[0,0,0],scale=255)

## 采用SAM提取，形成shp文件

In [ ]:
from samgeo import tms_to_geotiff , SamGeo # type: ignore
# from samgeo.hq_sam import SamGeo,show_image,download_file,overlay_images,tms_to_geotiff
sam = SamGeo(
    model_type="vit_h",
    automatic=False,
    sam_kwargs=None,
)
sam.set_image(imagePath)

# 基于BOX
AOI_Bound_Info = AOI_Bound.getInfo()
if box_fromGEE == True:
    boxes = [*AOI_Bound_Info['coordinates'][0][0],*AOI_Bound_Info['coordinates'][0][2]]
    sam.predict(boxes=boxes, point_crs="EPSG:4326", output="test_.tif", dtype="uint8")

## 可视化(可选)

In [ ]:
from shapely.geometry import box
import geopandas as gpd
import leafmap
boxes = [*AOI_Bound_Info['coordinates'][0][0],*AOI_Bound_Info['coordinates'][0][2]]
# Ensure that the 'box' function is called correctly to create geometries
gdf = gpd.GeoDataFrame(geometry=[box(*boxes)], crs="EPSG:4326")
m = leafmap.Map(center=Filter_bound.centroid().getInfo()['coordinates'][::-1], zoom=18, height="800px")
m.layers[-1].visible = False
m.add_raster(imagePath, layer_name="Image")
m.add_gdf(gdf, layer_name="Boxes")
m.add_raster("test_.tif", cmap="viridis", nodata=0, layer_name="Mask_box")
m

In [1]:
i= 1
save_path = "{}_ADMeanFused.tif".format(f'{i:05d}')

In [2]:
"{}_GLext.tif".format(f'{i:05d}')

'00001_GLext.tif'